In [1]:
import optuna
from optuna.samplers import TPESampler 

/home/dmh/biosofts/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn import svm, linear_model, ensemble
from sklearn import model_selection
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score

In [3]:
labels = pd.read_csv("antibacterial_lable.csv", index_col=0 )
data = pd.read_csv("mibig(antismash).csv", index_col=0 )

In [4]:
def get_sample_responses(vectors, responses):
    samples = vectors
    index = list(set(samples.index).intersection(set(labels.index)))
    responses = responses.loc[index]
    samples = samples.loc[index]
    return samples, responses

In [5]:
X , y = get_sample_responses(data, labels)
bgc_ids = pd.Series(y.index)

In [6]:
def objective(trial): 
    param = {
        "nu": trial.suggest_uniform('nu',0.0,0.5),
        "kernel": trial.suggest_categorical('kernel', ['poly','rbf','sigmoid']),
        "shrinking": trial.suggest_categorical('shrinking', [False, True]),
        "class_weight": trial.suggest_categorical('class_weight', ['balanced',None])
    }
    
    clf = svm.NuSVC(**param,probability = True)
    
    acc_kf = []
    
    cv = KFold(n_splits=5,random_state=0,shuffle=True)
    for i, (id_train, id_val) in enumerate(cv.split(bgc_ids)):
        train_ids, val_ids = bgc_ids[id_train], bgc_ids[id_val]
        X_train, X_val = X.loc[train_ids].values, X.loc[val_ids].values
        y_train, y_val = y.loc[train_ids].values, y.loc[val_ids].values
        #print('Train:', len(X_train), 'Test:', len(X_val))
    
        clf.fit(X_train, y_train)
        pred = clf.predict(X_val)
    
        acc = accuracy_score(pred, y_val)
        acc_kf.append(acc)
        
    acc_kf = np.array(acc_kf).mean()
    
    return acc_kf

In [7]:
study = optuna.create_study(direction="maximize",sampler=TPESampler(),pruner=optuna.pruners.HyperbandPruner())

[I 2024-07-01 04:38:53,409] A new study created in memory with name: no-name-58868b4a-d4cc-4db7-b0ad-a80956af3133


In [8]:
%%time
study.optimize(objective, n_trials=1)

[I 2024-07-01 04:40:04,433] Trial 0 finished with value: 0.720376984126984 and parameters: {'nu': 0.3927737021620289, 'kernel': 'rbf', 'shrinking': False, 'class_weight': None}. Best is trial 0 with value: 0.720376984126984.


CPU times: user 1min 10s, sys: 148 ms, total: 1min 10s
Wall time: 1min 10s
